In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("D4.ipynb")

# Evaluation Metrics & Model Selection (2 pts Possible)

Your learning objective in lab 4 is to understand the theory & practice of model selection. This is the most important topic we will cover this quarter. Specifically you will
- understand confusion matrices and classification metrics
- implement cross validation
- model selection w.r.t. lasso-regression
- (optional/supplemental) review of the bias-variance tradeoff

Fundamentally Model selection is about estimating how well your model will perform in the real world given some metric you care about. There are many model classes (linear regression, decision trees, neural networks, ...) and for every model class there are many possible choices, you the machine learning practitioner, can make (L1 vs. L2 regularization, the regularization strength, max tree depth, cost complexity,...). These choices often go beyond the scope of minimizing some error metric and require you to think about the nature of your problem, how it will be deployed, and the risks associated with different kinds of errors. Given the complex space of modeling decisions and the very real stakes of getting things wrong (false negative cancer test, false positive covid test, incorrectly denying someone a home loan, incorrectly approving someone of a home loan, etc...) it is paramount that you correctly estimate your models performance on (unseen) real world data and select your metrics according to real world stakes.

Machine learning without an understanding of model selection and evaluation metrics can cause real world harms!

In [ ]:
# !!!IMPORTANT!!! uncomment if needed for module installation
# Please comment out when you submit to gradescope

# %pip install -q --force-reinstall git+https://github.com/COGS118A/quiz_module.git
# %pip install -q otter-grader
# %pip install -U scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer, load_diabetes
from sklearn.metrics import f1_score, mean_squared_error
from sklearn.model_selection import train_test_split

from sklearn.metrics import zero_one_loss, accuracy_score, roc_auc_score, f1_score
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams['font.size'] = '16'

import warnings
warnings.filterwarnings('ignore')

rng = np.random.RandomState(0)

In [ ]:
# Setup for JupyterQuiz
from quiz import display_quiz, record_quiz, check_quiz_answer, show_chosen_option
from IPython.core.display import display, Javascript, HTML

D4_path = "https://raw.githubusercontent.com/COGS118A/DiscussionLabExercises/main/D4/"

## 1. Metrics (1.1 pts)

In this section of the lab, you will measure and analyze a binary classifier trained to detect breast cancer.

### Cancer Dataset

The UCI ML Breast Cancer Wisconsin (Diagnostic) dataset is a binary classificiation task generated from microscopic images of 569 biopsy samples. The class of each sample is either malignant or benign cancer. The variables are statistical summaries of attributesmeasured from the cell nuclei of each image. For instance, attributes include the area of a cell nucleus, shape measurements (like concavity of nucleus), grayscale texture, and many others. Each such attribute is measured for each cell nucleus in the image. For each attribute three statistical summaries are calculated to create the features: the mean, standard error, and worst (largest) measurement made across all the cell nuclei in the image.

For more details see https://scikit-learn.org/stable/datasets/toy_dataset.html#breast-cancer-dataset

In [ ]:
data_bcw  = load_breast_cancer()
df_bcw = pd.DataFrame( data_bcw['data'], columns=data_bcw['feature_names'] )

fig = plt.figure(figsize = (16,12))
ax = fig.gca()
df_bcw.hist(ax = ax);
plt.tight_layout();

### Binary Classifier

We will now train a binary classifier. Don't worry too much about the model details, your task will be to evaluate it. Our proceedure is as follows:

1. create a model pipeline which (1) preprocesses the data and (2) runs it through logistic regression (binary classifier)
2. split the data into train and test sets
3. train the model

If you are curious about a pipeline you can read more about them here on (sklearn's documentation page)[https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html]. In machine learning, there are transformations we might perform on data such as 1-hot-encoding, rescaling the data to have 0 mean and 1 variance. These transformations occur before we pass data to a model. A pipeline is a software abstraction that allows us to chain these transformations and a model together.

In [ ]:
# Think of this simply as a binary classifier. 
# Your job is to evaluate it.
classifier_model = make_pipeline(
    StandardScaler(), 
    LogisticRegression(random_state=0, penalty='l2')
)

# split our cancer dataset into a train and test
X_train_bcw, X_test_bcw, y_train_bcw, y_test_bcw = train_test_split(
    data_bcw['data'], data_bcw['target'], test_size=0.33, random_state=42)

classifier_model.fit(X_train_bcw, y_train_bcw)

y_pred_train_bcw = classifier_model.predict(X_train_bcw)
y_pred_test_bcw = classifier_model.predict(X_test_bcw)

### Q: 1a confusion matrices (0.3 pts)

A confusion matrix is a tool for measuring the occurrence of different kinds of correct and incorrect predictions in classification problems. In a binary classification problem (i.e. positive cancer prediction vs negative cancer prediction) your classifier may _incorrectly_ predict a `positive` cancer result in which case you have a `false positive` or your classifier may _incorrectly_ predict a `negative` cancer result in which case you have a `false negative`. Commonsense tells us the risks associated with these different errors are _very_ different: one increases the societal cost of healthcare the other _fails to detect cancer_ and may result in a preventable death. If you are familiar with statistics literature, you might know these as type I and type II errors. As a convention, the rows correspond to _actual_ classes and the columns correspond to _predicted_ classes.

Note the for binary classification the confusion matrix shows you 

|                 |                |
|-----------------|----------------|
| True Negatives | False Positives |
| False Negatives | True Positives |


**Question** Given the below confusion matrix, please compute the recall, precision, sensitivity, and f1 metrics

In [ ]:
cm = confusion_matrix(y_test_bcw, y_pred_test_bcw)

# the text version takes the true labels 
# and the predictions AND THE ORDER OF ARGUMENTS MATTERS!!
# don't mess it up

disp = ConfusionMatrixDisplay(cm);
disp.plot()

In [ ]:
(true_negatives, false_positives,
 false_negatives, true_positives
) = confusion_matrix(y_train_bcw, y_pred_train_bcw).ravel()

In [ ]:
def recall(true_positives, false_positives, true_negatives, false_negatives):
    """computes recall
    Arguments:
        true_p# END QUESTIONositives: int
            number of true positives
        false_positives: int
            number of false positives
        true_negatives: int
            number of true negatives
        false_negatives: int
            number of false negatives
    Returns:
        int
        recall
    """ 
    ...

In [ ]:
grader.check("recall")

In [ ]:
def precision(true_positives, false_positives, true_negatives, false_negatives):
    """computes precision
    Arguments:
        true_positives: int
            number of true positives
        false_positives: int
            number of false positives
        true_negatives: int
            number of true negatives
        false_negatives: int
            number of false negatives
    Returns:
        int
        precision
    """ 
    ...

In [ ]:
grader.check("precision")

In [ ]:
def harmonic_mean(a, b):
    """compute the harmonic mean...I think you will find this helpful!
    
    This mean is appropriate when you are taking the average of two rates, such as precision and recall.
    
    2*(a*b)/(a+b)
    
    https://en.wikipedia.org/wiki/Harmonic_mean
    Arguments:
        a: float
            okay...this ought to be a rational number.
        b: float
            okay...this ought to be a rational number.
    Returns:
        float
        harmonic_mean
    """ 
    ...
    
def f1(true_positives, false_positives, true_negatives, false_negatives):
    """computes f1. hint hint: its the harmonic mean of precision and recall
    Arguments:
        true_positives: int
            number of true positives
        false_positives: int
            number of false positives
        true_negatives: int
            number of true negatives
        false_negatives: int
            number of false negatives
    Returns:
        int
        f1
    """
    ...

In [ ]:
grader.check("f1_score")

### Metric Multiple Choice (0.5 pts)

In [ ]:
HTML(display_quiz(f"{D4_path}A.txt"))

In [ ]:
# show your choice
with open("record.txt", "a+") as r:
    pass
MCQ = ["A1", "A2"]
for q in MCQ:
    show_chosen_option(q)

### 1b plot ROC curve & compute AUC (0.3 pts)

The Reciever Operator Characteristic (ROC) characterizes binary classifiers by plotting the false positive rate against the false positive rate. How might a single trained model have multiple values for false positives and true positives you might ask? It turns out that vary what is called the decision threshold. In a decision tree for example, each node can report the probability of a class at a certain node, e.g., `P(cat|data) = 1/3` (read as 'probability of cat given our data'...or the probability our data point is a cat). You can set a decision threshold anywhere between 0 and 1. For our cat example, if the threshold for predicting `cat` is 1/4, then even though P(cat|data) < P(dog|data), we would predict `cat`. There are practical reasons for calibrating the decision threshold, e.g., if we wanted to build a database of every cat photo in the world we might accept a higher false positive rate (some dogs sneaking in) in exchange for maximizing the number of overall cats.

For more information check out [this wonderful github page](https://github.com/dariyasydykova/open_projects/tree/master/ROC_animation).

![img](https://github.com/dariyasydykova/open_projects/raw/master/ROC_animation/animations/cutoff.gif)

NOTE: Cognitive science has used ROC curves to characterize the performance of human sensory and memory systems. ROC applies to more than just computer algorithms. In fact, it was first used to test radar operators in WWII.

**Question** Given the classifier model `classifier_model` (defined in cells above) plot an ROC curve and compute the AUC.

REMEMBER:
$$FPR=\frac{FP}{FP+TN}$$
$$TPR=\frac{TP}{TP+FN}$$

AUC here is defined as the area under the ROC curve. (there are different ways to approximate this...think back to Reimann sums)

In [ ]:
# hint use classifier_model.decision_function(X_test_bcw) to get prediction scores
scores = classifier_model.predict_proba(X_test_bcw)[:,1]
fps = []
tps = []
for threshold in np.linspace(0,1,1000):
    y_pred = scores > threshold
    # Step 1: Compute the confusion matrix
    # Step 2: Compute the TP and FP 
    # and append it to fps and tps
    ...

In [ ]:
def my_auc(fps, tps):
    """my_auc computes the AUC of ROC. Feel free to convert these to numpy arrays.
    Arguments:
        fps: List[float]
            false positive rates
        tps: List[float]
            true positive rates
    Returns:
        float
        the area under the ROC curve
    """    
    ...

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(8,8))
ax.plot(fps, tps, linewidth=5, color='k')
ax.plot(np.linspace(0,1, 20), np.linspace(0,1, 20), linewidth=5, color='b', linestyle='--')
assert fps != [] and tps != []
ax.fill_between(fps, tps, [0]*len(tps), color='grey', alpha=0.3)
ax.set_xlabel("false positive rate")
ax.set_ylabel("true positive rate");

## 2. Cross Validation (0.9 pts)

Cross validation is a sampling based method for model selection. We first shuffle our data and then partition it into _k-folds_. Then we train $k$ models: each time we will use a fold to evaluate a model and train on the rest of the dataset. The primary goal of cross validation is to understand how well a model you train will generalize to (unseen) real world data. This also allows us to estimate the _variance_ or uncertainty of this prediction -- did each k-fold produce similar results or wildly different results? Think back to how the bias-variance relates.

In this section you will complete your own cross validation algorithm.
### **Q2.a (0.4 pts)**
Implement cross validation to estimate the genearlization f1-score of a provided decision tree. Below you are provided a code skeleton you need to flesh out.

In [ ]:
def permute(design_matrix, targets, rng):
    """Randomly permute (shuffle) your dataset!
    Arguments:
        design_matrix: np.ndarray
            an N by M matrix where N is the number of examples and M is the number of features  
        targets: np.ndarray
            a column vector of dependent variables
        rng:
            a random number generator
    Returns:
        Tuple[np.ndarray, np.ndarray]
        vectors sorted in unison, e.g., the rows are still paired
    """    
    assert design_matrix.shape[0] == targets.shape[0]
    permutation_indices = rng.permutation(len(design_matrix))
    return design_matrix[permutation_indices, :], targets[permutation_indices]
    
def kfold_cross_validation(X, y, model, metric, folds=7):
    """kfold_cross_validation performs leave 1 out cross validation on some model to estimate some metric
    
    Arguments:
        X: np.ndarray
            an N by M matrix where N is the number of examples and M is the number of features  
        targets: np.ndarray
            a column vector of dependent variables
        model:
            an sklearn model
        metric:
            an sklean metric
        folds: int
            number of folds to perform
    Returns:
        Tuple[np.ndarray, np.ndarray]
        vectors sorted in unison, e.g., the rows are still paired
    """   
    rng = np.random.RandomState(0)
    results = np.zeros(folds)
    
    #
    X, y = permute(X, y, rng)

    for k in range(folds):
        N = (len(X) // folds)
        start = k * N
        end = start + N
        if k == folds - 1 and N %2 == 1:
            end = start + N + 1
        # hint: np.delete might make your selection of training data much simpler
        # hint: for training data you need to select everything not in the fold
        # select a training X
        training_X = ...
        # select a training y
        training_y = ...
        # select a test X
        test_X = ...
        # select a test y
        test_y = ...
        fitted_model = model.fit(training_X, training_y) 
        pred_y = fitted_model.predict(test_X) 
        # metric
        f1_score = ...
        results[k] = f1_score
    return results

In [ ]:
# here you will run your code. You don't need to edit anything here

data  = load_breast_cancer()
X = data["data"]
y = data["target"]
model = DecisionTreeClassifier(max_depth=3, random_state=0)
results = kfold_cross_validation(X, y, model, f1_score, folds=7)
mean_f1 = results.mean()
std_f1 = results.std()
print(f"f1-score: {mean_f1:.3f}+-{std_f1:.3f}")
df = pd.DataFrame(results, columns=["f1-score"])
df.index.name = "fold"
df

How did the model perform across these k-folds? Did cross validation yield similar results? Should we be confident in our estimation of the models error?

### 3. L1 Loss Feature Selection (0.5 pts)

Think back to regression, loss functions & regularization. If you only considered linear regression, you would already have quite a few modeling decisions to make: what loss function do I use? what regularization type do I use? If I do choose regularization, how strongly should I regularize my model?

In this section, you will answer this for yourself. Specifically, you are going to see how the choice of regularization strength affects a model qualitatively (number of non-zero coefficients) and quantitatively (error on a heldout dataset).

The model you will analyze is Lasso regression. This model uses a squared l2 error term and an l1 regularization term.

$$||y - Xw||^2_2 + \lambda ||W||_1$$

In other words this is just ordinary least squares _but_ in addition to error we also punish models in which `sum(W)` is large. Think about what this does to the gradient (or see for yourself in the demo below).

In [ ]:
import itertools

def plot_contour(f, x1bound, x2bound, resolution, ax):
    x1range = np.linspace(x1bound[0], x1bound[1], resolution)
    x2range = np.linspace(x2bound[0], x2bound[1], resolution)
    xg, yg = np.meshgrid(x1range, x2range)
    zg = np.zeros_like(xg)
    for i,j in itertools.product(range(resolution), range(resolution)):
        zg[i,j] = f([xg[i,j], yg[i,j]])
    ax.contour(xg, yg, zg, 30)
    
    # Move left y-axis and bottim x-axis to centre, passing through (0,0)
    ax.spines['left'].set_position('center')
    ax.spines['bottom'].set_position('center')

    # Eliminate upper and right axes
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    mini, minj = np.unravel_index(np.argmin(zg), zg.shape)
    ax.scatter(xg[mini, minj], yg[mini, minj], marker='o', color='k', s=100)
    return ax, (xg[mini, minj], yg[mini, minj])

rng2 = np.random.RandomState(1234)
N = 30
xs = rng2.rand(N) * 10
ys = -5*xs + 10 + 8*rng2.randn(N)

REGULARIZATION_STRENGTHS = [10, 100, 500, 1000] # play with this parameter!!!

def plot_gradient(REGULARIZATION_STRENGTH):
    cost = lambda w: np.mean((w[0] + w[1]*xs - ys)**2)
    l1_penalty = lambda w: np.sum(np.abs(w))
    cost_plus_l1 = lambda w: cost(w) + REGULARIZATION_STRENGTH*l1_penalty(w)

    # ignore this ugly plotting code
    fig, axes = plt.subplots(ncols=3, nrows=1, constrained_layout=True, figsize=(15, 5))
    _, minpnt1 = plot_contour(cost, [-20,20], [-20,20], 200, axes[0])
    axes[0].set_title("∇l2(y - Wx)**2")
    plot_contour(l1_penalty, [-20,20], [-20,20], 200, axes[1])
    axes[1].set_title("∇l1(W)")
    _, minpnt2 = plot_contour(cost_plus_l1, [-20,20], [-20,20], 200, axes[2])
    axes[2].set_title("∇ l2(y - Wx)**2 + lambda*l1(W)")
    axes[2].scatter(*minpnt1, s=100, color='k', alpha=0.5, marker='o')
    axes[2].arrow(*minpnt1, (minpnt2[0]-minpnt1[0])*.8, (minpnt2[1]-minpnt1[1])*.8, head_width = 1)
    print((f"REGULARIZATION_STRENGTH = {REGULARIZATION_STRENGTH}"))
    plt.show()
    
_ = [plot_gradient(strength) for strength in REGULARIZATION_STRENGTHS]

Anyhow, the choice of $\lambda$ has significant consequences on your model. One way we can tune such hyperparameters is by systematically trying out various settings and comparing the results. We can do this with gridsearch where we try a range of values.

**Question 3a (0.3 pts)** perform a grid search over possible regularization strengths to see which value produces a good model. You are provided some skeleton code you need to flesh out, in particular `number_of_non_zero_coefficients`. Also, while we typically write the regularization strength as $\lambda$ (if your are curious why this is and like advanced math or physics...look up Lagrangians) but in sklearn's Lasso implementation, this parameter is called `alpha`.

Hint: you should use `np.isclose` when comparing floats!

In [ ]:
data2 = load_diabetes()
X2 = data2["data"]
y2 = data2["target"]

(X2_train, X2_test, 
 y2_train, y2_test
) = train_test_split(X2, y2, test_size=0.33, random_state=42)

In [ ]:
def get_coefficients(lasso_pipeline_model):
    """get the vector of coefficients of the model
    
    Arguments:
        lasso_pipeline_model: sklearn pipeline
    Returns:
        np.array
        vector of weights (coefficients)
    """  
    return lasso_pipeline_model.steps[1][1].coef_

def number_of_non_zero_coefficients(lasso_pipeline_model):
    """compute mean squared error
    
    Arguments:
        y_true: np.array
            true response variable
        y_pred: np.array
            predicted response variable
    Returns:
        float
        mean squared error
    """  
    ...

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

alphas = np.linspace(0.01,2.5,20)
non_zero_coefficients = []
mean_squared_errors = []
for alpha in alphas:
    lasso_pipeline_model = make_pipeline(StandardScaler(), Lasso(random_state=0, alpha=alpha))
    model = lasso_pipeline_model.fit(X2_train, y2_train)
    non_zero_coefficients.append(
        number_of_non_zero_coefficients(model)
    )
    mean_squared_errors.append(
        r2_score(y2_test, model.predict(X2_test))
    )
    
    
fig, ax = plt.subplots(1, 1, figsize=(16,6))
ax.plot(alphas, non_zero_coefficients, linewidth=5, color='k', label="number of non-zero coefficients")
ax2 = ax.twinx()
ax2.plot(alphas, mean_squared_errors, linewidth=5, color='r', label="$r^2$")
ax.legend(loc='upper left')
ax2.legend(loc='upper right');
ax.set_xlabel("Regularization strength: call this alpha or $\lambda$")
ax.set_ylabel("# of non-zero coefficients")
ax2.set_ylabel("$r^2$");

### Regularization Quiz (0.2 pts)

In [ ]:
HTML(display_quiz(f"{D4_path}B.txt"))

In [ ]:
# show your choice
with open("record.txt", "a+") as r:
    pass
MCQ = ["B1"]
for q in MCQ:
    show_chosen_option(q)

In [ ]:
%%javascript
IPython.notebook.save_notebook()

# Submission Guidelines
Congratulations! You have completed this discussion lab.

Have a look back over your answers, and also make sure to `Restart & Run All` from the kernel menu to double check that everything is working properly. This restarts everything and runs your code from top to bottom.

When you are ready to submit your assignment, you can click `Validate` at the top. Note that in some assignments the code will take too long to run and validation may fail. Validation is just a final check that all the asserts are passing without failing.

Once you're happy with your work, click the disk icon to save, and submit the zip file onto gradescope. **You MUST submit all the required component to receive credit.**

Note that you can submit at any time, but **we grade your most recent submission**. This means that **if you submit an updated notebook after the submission deadline, it will be marked as late**.